# Video Classification

This script consists all the steps for video classification.

1. extracting the images from the nii files.
2. converting the images of the nii file into the video.
3. converting the video into images again.
4. running extract_features.py to get the features from the vgg.
5. running train.py to train the deep neural model.



In [1]:
import os
import numpy as np
import nibabel as nib
import shutil
import glob
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import cv2
import matplotlib.image as mpimg
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score
from skimage import filters
from skimage import measure
from skimage.color import rgb2gray
from skimage.morphology import closing, square

## Converting the nii files into images

In [ ]:
classes = ['Alzheimer',"MCI", 'Normal']

home_files_dir = '/home/ubuntu/Select_original_fmri/'

data = []
labels = []

try:
    shutil.rmtree(home_files_dir + 'images/')
except:
    print("No images folder in directory")
    pass

try:
    os.mkdir(home_files_dir + 'images/')
except:
    pass
        
count = 1
for class_ in classes:
    print ('working on ' + class_ + '...')
    for root, dir ,files in os.walk(os.path.join(home_files_dir2 , class_)):      
        try:
            os.mkdir(home_files_dir + 'images/' + class_ + '/')
        except:
            pass
        
        for file_ in files:
            print 'working on ' + file_ + '...'
            full_path = home_files_dir + 'images/' + class_ + '/' + file_ + '/'
            try:
                os.mkdir(full_path)
            except:
                continue
            try:
                count = 0
                x = nib.load(os.path.join(home_files_dir + 'niifiles/' , class_) + '/' + file_).get_data()
                for i in xrange(x.shape[3]):
                    for j in xrange(x.shape[2]):
                            y = x[:, :, j, i]
                            img = Image.fromarray(y)
                            img = img.convert("RGB")
                            img = img.resize([224, 224])
                            img.save(full_path + 'i_' + str(count) + ".jpg")
                            count+=1
            except:
                print file_
    #os.system('rm -rf %s%s' %(home_files_dir2, class_))

No images folder in directory
working on Alzheimer...
working on A25_018_S_5074.nii...
working on A27_018_S_4696.nii...
working on A78_019_S_4549.nii...
working on A15_100_S_5106.nii...
working on A41_130_S_4982.nii...
working on A48_130_S_4984.nii...
working on A16_053_S_5208.nii...
working on A1_136_S_4993.nii...
working on A57_130_S_5006.nii...
working on A20_019_S_5012.nii...
working on A19_019_S_5019.nii...
working on A93_130_S_4589.nii...
working on A33_006_S_4153.nii...
working on A58_006_S_4867.nii...
working on A29_010_S_5163.nii...
working on A34_002_S_5018.nii...
working on A90_053_S_5070.nii...
working on A26_018_S_4733.nii...
working on A98_130_S_4660.nii...
working on A53_006_S_4546.nii...
working on A63_013_S_5071.nii...
working on A74_019_S_4477.nii...
working on A55_130_S_4990.nii...
working on A3_130_S_5231.nii...
working on A95_130_S_4641.nii...
working on A72_130_S_5059.nii...
working on A104_130_S_4971.nii...
working on A11_130_S_4730.nii...
working on A18_031_S_40

In [24]:
try:
    # removing the completed directory in the videoclassification/data directory.
    shutil.rmtree("/home/ubuntu/final_src/AlzheimerDetection/videoclassification/data/completed")
except:
    pass

for class_ in classes:
    try:
        os.mkdir(os.path.join("/home/ubuntu/final_src/AlzheimerDetection/videoclassification/data/videos/" , class_))
    except:
        shutil.rmtree(os.path.join("/home/ubuntu/final_src/AlzheimerDetection/videoclassification/data/videos/" , class_))
        os.mkdir(os.path.join("/home/ubuntu/final_src/AlzheimerDetection/videoclassification/data/videos/" , class_))
        
    for root, dirs ,files in os.walk(os.path.join(home_files_dir + "images/" , class_)):
        for dir_ in dirs:
            print 'working on ' + dir_ + "..." + class_
            save_dir = '/home/ubuntu/final_src/AlzheimerDetection/videoclassification/data/videos/' + class_ + '/' + dir_ + '.avi'
            print os.system("avconv  -i " + home_files_dir + 'images/' + class_ + '/' + dir_ + '/i_%d.jpg' + " -c:v libx264 " + save_dir)
try:
    os.mkdir("/home/ubuntu/final_src/AlzheimerDetection/videoclassification/data/completed")
except:
    pass

working on A25_018_S_5074.nii...Alzheimer
0
working on A27_018_S_4696.nii...Alzheimer
0
working on A78_019_S_4549.nii...Alzheimer
0
working on A15_100_S_5106.nii...Alzheimer
0
working on A41_130_S_4982.nii...Alzheimer
0
working on A48_130_S_4984.nii...Alzheimer
0
working on A16_053_S_5208.nii...Alzheimer
0
working on A1_136_S_4993.nii...Alzheimer
0
working on A57_130_S_5006.nii...Alzheimer
0
working on A20_019_S_5012.nii...Alzheimer
0
working on A19_019_S_5019.nii...Alzheimer
0
working on A93_130_S_4589.nii...Alzheimer
0
working on A33_006_S_4153.nii...Alzheimer
0
working on A58_006_S_4867.nii...Alzheimer
0
working on A29_010_S_5163.nii...Alzheimer
0
working on A34_002_S_5018.nii...Alzheimer
0
working on A90_053_S_5070.nii...Alzheimer
0
working on A26_018_S_4733.nii...Alzheimer
0
working on A98_130_S_4660.nii...Alzheimer
0
working on A53_006_S_4546.nii...Alzheimer
0
working on A63_013_S_5071.nii...Alzheimer
0
working on A74_019_S_4477.nii...Alzheimer
0
working on A55_130_S_4990.nii...A

In [25]:
path = "/home/ubuntu/final_src/AlzheimerDetection/videoclassification/data/images/"
dst = "/home/ubuntu/final_src/AlzheimerDetection/videoclassification/data/"
files_list = [] 
test_list = []
train_list = []

count  = 0
for class_ in classes:
    print class_
    try:
        os.mkdir(dst+"test/"+class_)
        os.mkdir(dst+"train/"+class_)
    except:
        shutil.rmtree(dst+"train/"+class_)
        shutil.rmtree(dst+"test/"+class_)
        os.mkdir(dst+"test/"+class_)
        os.mkdir(dst+"train/"+class_)
    for root,dirs,files in os.walk(os.path.join(path,class_)):
        for f in files:
            files_list.append(f)
        test = int(len(files_list)*.1)
        train = int(round(len(files_list)*.9))
        test_list += list(files_list[:test])
        train_list += list(files_list[test:train])
        for i in test_list:
            shutil.move(src=path+class_ + i, dst=dst+"test/"+class_+i)
        for i in train_list:
            shutil.move(src=path +  class_ + i, dst=dst+"train/" +class_+i)
        files_list = []
        train_list = []
        test_list = []


Alzheimer/
MCI/
Normal/


In [13]:
% cd test_codes/data/

/home/ubuntu/test_codes/data


In [ ]:
!python 2_extract_files.py


ffmpeg version 3.3.2 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 4.8 (Ubuntu 4.8.4-2ubuntu1~14.04.3)
  configuration: --extra-libs=-ldl --prefix=/opt/ffmpeg --mandir=/usr/share/man --enable-avresample --disable-debug --enable-nonfree --enable-gpl --enable-version3 --enable-libopencore-amrnb --enable-libopencore-amrwb --disable-decoder=amrnb --disable-decoder=amrwb --enable-libpulse --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libx265 --enable-libfdk-aac --enable-libvorbis --enable-libtheora --enable-libmp3lame --enable-libopus --enable-libvpx --enable-libspeex --enable-libass --enable-avisynth --enable-libsoxr --enable-libxvid --enable-libvidstab --enable-libwavpack --enable-nvenc
  libavutil      55. 58.100 / 55. 58.100
  libavcodec     57. 89.100 / 57. 89.100
  libavformat    57. 71.100 / 57. 71.100
  libavdevice    57.  6.100 / 57.  6.100
  libavfilter     6. 82.100 /  6. 82.100
  libavresample   3.  5.  0 /  3.  5.  0
  libswscale      4.  6.100

frame= 6300 fps=3032 q=24.8 Lsize=N/A time=00:04:12.08 bitrate=N/A speed= 121x    
video:17155kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
Generated 6300 frames for M45_018_S_2138
ffmpeg version 3.3.2 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 4.8 (Ubuntu 4.8.4-2ubuntu1~14.04.3)
  configuration: --extra-libs=-ldl --prefix=/opt/ffmpeg --mandir=/usr/share/man --enable-avresample --disable-debug --enable-nonfree --enable-gpl --enable-version3 --enable-libopencore-amrnb --enable-libopencore-amrwb --disable-decoder=amrnb --disable-decoder=amrwb --enable-libpulse --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libx265 --enable-libfdk-aac --enable-libvorbis --enable-libtheora --enable-libmp3lame --enable-libopus --enable-libvpx --enable-libspeex --enable-libass --enable-avisynth --enable-libsoxr --enable-libxvid --enable-libvidstab --enable-libwavpack --enable-nvenc
  libavutil      55. 58.100 / 55. 58.100
  libavco

frame= 6300 fps=2756 q=24.8 Lsize=N/A time=00:04:12.08 bitrate=N/A speed= 110x    
video:17669kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
Generated 6300 frames for M300_130_S_4405
ffmpeg version 3.3.2 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 4.8 (Ubuntu 4.8.4-2ubuntu1~14.04.3)
  configuration: --extra-libs=-ldl --prefix=/opt/ffmpeg --mandir=/usr/share/man --enable-avresample --disable-debug --enable-nonfree --enable-gpl --enable-version3 --enable-libopencore-amrnb --enable-libopencore-amrwb --disable-decoder=amrnb --disable-decoder=amrwb --enable-libpulse --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libx265 --enable-libfdk-aac --enable-libvorbis --enable-libtheora --enable-libmp3lame --enable-libopus --enable-libvpx --enable-libspeex --enable-libass --enable-avisynth --enable-libsoxr --enable-libxvid --enable-libvidstab --enable-libwavpack --enable-nvenc
  libavutil      55. 58.100 / 55. 58.100
  libavc

frame= 6300 fps=2698 q=24.8 Lsize=N/A time=00:04:12.08 bitrate=N/A speed= 108x    
video:18249kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
Generated 6300 frames for M30_130_S_2403
ffmpeg version 3.3.2 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 4.8 (Ubuntu 4.8.4-2ubuntu1~14.04.3)
  configuration: --extra-libs=-ldl --prefix=/opt/ffmpeg --mandir=/usr/share/man --enable-avresample --disable-debug --enable-nonfree --enable-gpl --enable-version3 --enable-libopencore-amrnb --enable-libopencore-amrwb --disable-decoder=amrnb --disable-decoder=amrwb --enable-libpulse --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libx265 --enable-libfdk-aac --enable-libvorbis --enable-libtheora --enable-libmp3lame --enable-libopus --enable-libvpx --enable-libspeex --enable-libass --enable-avisynth --enable-libsoxr --enable-libxvid --enable-libvidstab --enable-libwavpack --enable-nvenc
  libavutil      55. 58.100 / 55. 58.100
  libavco

frame= 6300 fps=3296 q=24.8 Lsize=N/A time=00:04:12.08 bitrate=N/A speed= 132x    
video:16564kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
Generated 6300 frames for M2_012_S_4012
ffmpeg version 3.3.2 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 4.8 (Ubuntu 4.8.4-2ubuntu1~14.04.3)
  configuration: --extra-libs=-ldl --prefix=/opt/ffmpeg --mandir=/usr/share/man --enable-avresample --disable-debug --enable-nonfree --enable-gpl --enable-version3 --enable-libopencore-amrnb --enable-libopencore-amrwb --disable-decoder=amrnb --disable-decoder=amrwb --enable-libpulse --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libx265 --enable-libfdk-aac --enable-libvorbis --enable-libtheora --enable-libmp3lame --enable-libopus --enable-libvpx --enable-libspeex --enable-libass --enable-avisynth --enable-libsoxr --enable-libxvid --enable-libvidstab --enable-libwavpack --enable-nvenc
  libavutil      55. 58.100 / 55. 58.100
  libavcod

...

frame= 6300 fps=2888 q=24.8 Lsize=N/A time=00:04:12.08 bitrate=N/A speed= 116x    
video:19523kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
Generated 6300 frames for N165_031_S_4021
Generated 6720 frames for M283_031_S_4203
Generated 6720 frames for M127_006_S_4713
Generated 6720 frames for M325_053_S_4813
Generated 6720 frames for M144_012_S_4128
Generated 6720 frames for M171_013_S_4236
Generated 6720 frames for M273_031_S_4149
Generated 6720 frames for M161_013_S_1186
Generated 6720 frames for M188_013_S_4917
Generated 6720 frames for M38_053_S_2357
Generated 6720 frames for M13_002_S_4521
Generated 6720 frames for M288_031_S_4476
Generated 6720 frames for M43_018_S_2180
Generated 6720 frames for M324_130_S_4294
Generated 6720 frames for M183_130_S_4542
Generated 6720 frames for M50_002_S_0729
Generated 6720 frames for M226_019_S_4285
Generated 6720 frames for M102_006_S_4346
Generated 6720 frames for M20_002_S_4171
Generated 6720 frames for

Generated 6300 frames for M273_031_S_4149
ffmpeg version 3.3.2 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 4.8 (Ubuntu 4.8.4-2ubuntu1~14.04.3)
  configuration: --extra-libs=-ldl --prefix=/opt/ffmpeg --mandir=/usr/share/man --enable-avresample --disable-debug --enable-nonfree --enable-gpl --enable-version3 --enable-libopencore-amrnb --enable-libopencore-amrwb --disable-decoder=amrnb --disable-decoder=amrwb --enable-libpulse --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libx265 --enable-libfdk-aac --enable-libvorbis --enable-libtheora --enable-libmp3lame --enable-libopus --enable-libvpx --enable-libspeex --enable-libass --enable-avisynth --enable-libsoxr --enable-libxvid --enable-libvidstab --enable-libwavpack --enable-nvenc
  libavutil      55. 58.100 / 55. 58.100
  libavcodec     57. 89.100 / 57. 89.100
  libavformat    57. 71.100 / 57. 71.100
  libavdevice    57.  6.100 / 57.  6.100
  libavfilter     6. 82.100 /  6. 82.100
  libavresample   3.  5.  

frame= 6300 fps=2853 q=24.8 Lsize=N/A time=00:04:12.08 bitrate=N/A speed= 114x    
video:16889kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
Generated 6300 frames for A95_130_S_4641
ffmpeg version 3.3.2 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 4.8 (Ubuntu 4.8.4-2ubuntu1~14.04.3)
  configuration: --extra-libs=-ldl --prefix=/opt/ffmpeg --mandir=/usr/share/man --enable-avresample --disable-debug --enable-nonfree --enable-gpl --enable-version3 --enable-libopencore-amrnb --enable-libopencore-amrwb --disable-decoder=amrnb --disable-decoder=amrwb --enable-libpulse --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libx265 --enable-libfdk-aac --enable-libvorbis --enable-libtheora --enable-libmp3lame --enable-libopus --enable-libvpx --enable-libspeex --enable-libass --enable-avisynth --enable-libsoxr --enable-libxvid --enable-libvidstab --enable-libwavpack --enable-nvenc
  libavutil      55. 58.100 / 55. 58.100
  libavco

frame= 6300 fps=3014 q=24.8 Lsize=N/A time=00:04:12.08 bitrate=N/A speed= 121x    
video:16873kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
Generated 6300 frames for M161_013_S_1186
ffmpeg version 3.3.2 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 4.8 (Ubuntu 4.8.4-2ubuntu1~14.04.3)
  configuration: --extra-libs=-ldl --prefix=/opt/ffmpeg --mandir=/usr/share/man --enable-avresample --disable-debug --enable-nonfree --enable-gpl --enable-version3 --enable-libopencore-amrnb --enable-libopencore-amrwb --disable-decoder=amrnb --disable-decoder=amrwb --enable-libpulse --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libx265 --enable-libfdk-aac --enable-libvorbis --enable-libtheora --enable-libmp3lame --enable-libopus --enable-libvpx --enable-libspeex --enable-libass --enable-avisynth --enable-libsoxr --enable-libxvid --enable-libvidstab --enable-libwavpack --enable-nvenc
  libavutil      55. 58.100 / 55. 58.100
  libavc

frame= 6300 fps=3270 q=24.8 Lsize=N/A time=00:04:12.08 bitrate=N/A speed= 131x    
video:16467kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
Generated 6300 frames for N149_18_S_4399
ffmpeg version 3.3.2 Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 4.8 (Ubuntu 4.8.4-2ubuntu1~14.04.3)
  configuration: --extra-libs=-ldl --prefix=/opt/ffmpeg --mandir=/usr/share/man --enable-avresample --disable-debug --enable-nonfree --enable-gpl --enable-version3 --enable-libopencore-amrnb --enable-libopencore-amrwb --disable-decoder=amrnb --disable-decoder=amrwb --enable-libpulse --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libx265 --enable-libfdk-aac --enable-libvorbis --enable-libtheora --enable-libmp3lame --enable-libopus --enable-libvpx --enable-libspeex --enable-libass --enable-avisynth --enable-libsoxr --enable-libxvid --enable-libvidstab --enable-libwavpack --enable-nvenc
  libavutil      55. 58.100 / 55. 58.100
  libavco

In [20]:
% cd ..

/home/ubuntu/test_codes


In [21]:
!python extract_features.py

Using TensorFlow backend.
Traceback (most recent call last):
  File "extract_features.py", line 16, in <module>
    from data import DataSet
  File "/home/ubuntu/test_codes/data.py", line 12, in <module>
    from processor import process_image
  File "/home/ubuntu/test_codes/processor.py", line 4, in <module>
    from keras.preprocessing.image import img_to_array, load_img
  File "/usr/local/lib/python2.7/dist-packages/keras/__init__.py", line 3, in <module>
    from . import utils
  File "/usr/local/lib/python2.7/dist-packages/keras/utils/__init__.py", line 6, in <module>
    from . import conv_utils
  File "/usr/local/lib/python2.7/dist-packages/keras/utils/conv_utils.py", line 3, in <module>
    from .. import backend as K
  File "/usr/local/lib/python2.7/dist-packages/keras/backend/__init__.py", line 83, in <module>
    from .tensorflow_backend import *
  File "/usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py", line 1, in <module>
    import tensorflow as 

In [ ]:
!python train.py